In [3]:
from dotenv import load_dotenv
load_dotenv(override=True)

import json
from langchain_community.document_loaders import TextLoader
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from langchain.docstore.document import Document

Tworzenie dokumentów na podstawie fragmentów plików (ang.chunk):

In [4]:
loader = TextLoader('docs.md')
doc = loader.load()

# podział pliku na dokumenty według podwójnego znkau nwoej linii
documents = [Document(page_content=content) for content in doc[0].page_content.split("\n\n")]

# przetworzenie dokumentów promptem opisującym ich źródło (w tym przypadku jest to wybór imienia osoby, z którą powiązany jest dany dokument)
model = ChatOpenAI()

system_message = SystemMessage('''Describe the following document with one of the following keywords:
            Mateusz, Jakub, Adam. Return the keyword and nothing else.''')

descriptions = []
for document in documents:
    description = model.invoke([
        system_message,
        HumanMessage(f"{document.page_content}")
    ])
    descriptions.append(description.content)

for index, description in enumerate(descriptions):
    # Zaktualizuj 'source' w 'metadata' odpowiedniego dokumentu za pomocą opisu
    documents[index].metadata['source'] = description

# Przekształć listę obiektów 'documents' na listę słowników
documents_dict = [{"page_content": doc.page_content, "metadata": doc.metadata} for doc in documents]

try:
    with open("docs.json", "w", encoding='utf8') as file:
        json.dump(documents_dict, file, indent=1)
except FileNotFoundError:
    print("Upewnij się, że ścieżka jest poprawna.")